### Задание по программированию: Анализ текстов

In [109]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

#### 1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос"и "атеизм"(инструкция приведена выше).

In [125]:
newsgroups = datasets.fetch_20newsgroups(subset = 'all', categories = ['alt.atheism', 'sci.space'])

In [126]:
corpus = newsgroups.data
y = newsgroups.target

#### 2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [22]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [29]:
str(vectorizer.get_feature_names())

"[u'00', u'000', u'0000', u'00000', u'000000', u'000021', u'000050', u'000062david42', u'000406', u'000410', u'00041032', u'0004136', u'00041555', u'0004244402', u'0004246', u'00043819', u'0004422', u'00044513', u'00044808', u'00044939', u'0004651657', u'0004847546', u'0004988', u'0005', u'0005169', u'0008512', u'00090711', u'000th', u'001125', u'0011265', u'0012', u'001326', u'00140', u'001428', u'001442', u'001555', u'001718', u'001757', u'0018', u'0020', u'0022', u'002214', u'002341', u'0028', u'0029', u'0033', u'0034', u'003719', u'004119', u'004311', u'004405', u'006', u'0065', u'0098', u'00pm', u'01', u'010', u'0100', u'0100lines', u'010116', u'010326', u'010821', u'010847', u'011255', u'012536', u'013034', u'013423tan102', u'013657', u'014305', u'014506', u'014554', u'01463', u'015', u'015922', u'015931', u'01609', u'01776', u'01826', u'018b', u'0192', u'0195', u'01wb', u'02', u'020021', u'020259', u'020359', u'020504', u'020546', u'020701tan102', u'02115', u'02138', u'02139', u

In [127]:
X.shape

(1786, 28382)

#### 3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [38]:
param_grid = {'C': np.power(10.0, np.arange(-5, 6))}

cross_validator = KFold(n_splits = 5, shuffle = True, random_state = 241)
model = SVC(kernel = "linear", random_state = 241)
grid_search = GridSearchCV(model, param_grid, scoring = "accuracy", cv = cross_validator)

grid_search.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

#### После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант.

#### a.mean_validation_score — оценка качества по кросс-валидации
#### a.parameters — значения параметров

In [52]:
grid_search.best_score_

0.9932810750279956

In [128]:
grid_search.best_params_

{'C': 1.0}

####  4. Обучите SVM по всей выборке с лучшим параметром C, найденным на предыдущем шаге.

In [129]:
best_estimator = grid_search.best_estimator_

#### 5. Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. Укажите их через запятую, в нижнем регистре, в лексикографическом порядке.

In [138]:
words = vectorizer.get_feature_names()
coef = pd.DataFrame(best_estimator.coef_.data, best_estimator.coef_.indices)
top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending = False).head(10).index.map(lambda i: words[i])
top_words_sorted = top_words.sort_values()

In [139]:
','.join(top_words_sorted.tolist())

u'atheism,atheists,bible,god,keith,moon,religion,sci,sky,space'